# Imports

In [1]:
import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
from tqdm import tqdm
import math

from tqdm import tqdm
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from copy import deepcopy

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM
from sklearn.base import clone

from scipy.spatial.distance import cdist
from scipy.stats import ks_2samp
from scipy.optimize import minimize
from scipy.stats import wasserstein_distance

from sklearn.metrics import average_precision_score

from k_means_constrained import KMeansConstrained

/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/deepod2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/conda/lib/python3.11/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.11/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/deepod2/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/deepod2/lib/python3.11/site-packages/traitlets/config/application.py", line 1075

In [4]:
X_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')

X_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

y = np.where(y == 4, 0, 1)

In [5]:
len(X_train), len(y_train), len(X_test), len(y_test)

(955571, 955571, 238893, 238893)

# Setup

In [6]:
def create_phi(normal_data, c):
    """
    Concept creation function for normal data.
    Uses k-Means clustering to partition normal data into c clusters.
    
    Args:
        normal_data (numpy array): The normal data points.
        c (int): Number of desired normal concepts.
    
    Returns:
        list of numpy arrays: List of normal clusters.
    """
    kmeans = KMeansConstrained(n_clusters=c, size_min=20, random_state=42)
    labels = kmeans.fit_predict(normal_data)
    
    normal_concepts = [normal_data[labels == i] for i in range(c)]
    print("Finished creating normal concepts")
    
    return normal_concepts


def create_gamma(anomaly_data, c):
    """
    Concept creation function for anomaly data.
    Uses k-Means clustering to partition anomaly data into c clusters.
    
    Args:
        anomaly_data (numpy array): The anomaly data points.
        c (int): Number of desired anomaly concepts.
    
    Returns:
        list of numpy arrays: List of anomaly clusters.
    """
    kmeans = KMeansConstrained(n_clusters=c, size_min=20, random_state=42)
    labels = kmeans.fit_predict(anomaly_data)
    
    anomaly_concepts = [anomaly_data[labels == i] for i in range(c)]
    print("Finished creating anomaly concepts")
    
    return anomaly_concepts
    
def match_lambda(anomaly_concepts, normal_concepts):
    """
    Matches each normal concept with the closest anomaly concept.
    Uses Euclidean distance to determine the best match.
    
    Args:
        anomaly_concepts (list of numpy arrays): List of anomaly clusters.
        normal_concepts (list of numpy arrays): List of normal clusters.
    
    Returns:
        list of tuples: Pairs of (normal_concept, matched_anomaly_concept)
    """
    pairs = []
    remaining_anomalies = anomaly_concepts.copy()

    for normal_concept in normal_concepts:
        normal_centroid = np.mean(normal_concept, axis=0)
        anomaly_centroids = [np.mean(ac, axis=0) for ac in remaining_anomalies]

        distances = cdist([normal_centroid], anomaly_centroids, metric='euclidean')[0]
        closest_idx = np.argmin(distances)

        pairs.append((normal_concept, remaining_anomalies[closest_idx]))
        remaining_anomalies.pop(closest_idx)

    print("Finished matching concept pairs")
    
    return pairs

def lifelong_roc_auc(R):
    """
    Computes the Lifelong ROC-AUC metric.
    
    Args:
        R (numpy array): NxN matrix of ROC-AUC scores, where R[i, j] is the model's 
                         performance on concept j after learning concept i.
    
    Returns:
        float: Lifelong ROC-AUC score.
    """
    N = R.shape[0]
    lower_triangular_sum = np.sum(np.tril(R))
    normalization_factor = (N * (N + 1)) / 2

    return lower_triangular_sum / normalization_factor

def BWT(R):
    """
    Computes the Backward Transfer (BWT) score.
    
    Args:
        R (numpy array): NxN results matrix.
    
    Returns:
        float: BWT score.
    """
    N = R.shape[0]
    backward_transfer = 0
    count = 0

    for i in range(1, N):
        for j in range(i):
            backward_transfer += (R[i, j] - R[j, j])
            count += 1

    return backward_transfer / count if count > 0 else 0

def FWT(R):
    """
    Computes the Forward Transfer (FWT) score.
    
    Args:
        R (numpy array): NxN results matrix.
    
    Returns:
        float: FWT score.
    """
    N = R.shape[0]
    forward_transfer = 0
    count = 0

    for i in range(N):
        for j in range(i + 1, N): 
            forward_transfer += R[i, j]
            count += 1

    return forward_transfer / count if count > 0 else 0 

def kolmogorov_smirnov_test(X_old, X_new, alpha=0.05):
    """Detect concept drift using KS-test on feature distributions."""
    
    p_values = [ks_2samp(X_old[:, i], X_new[:, i]).pvalue for i in range(X_old.shape[1])]
    return np.any(np.array(p_values) < alpha)

def histogram_binning(X, bins=25):
    """Convert sample distributions into histograms."""
    
    return np.array([np.histogram(X[:, i], bins=bins, density=True)[0] for i in range(X.shape[1])]).T

def kl_divergence(P, Q):
    """Compute KL divergence between two distributions."""
    
    P, Q = np.clip(P, 1e-10, None), np.clip(Q, 1e-10, None)  # Avoid log(0)
    return np.sum(P * np.log(P / Q))

def strategic_sample_selection(X_old, X_new, top_k=100, learning_rate=0.01, num_iterations=100):
    """
    Selects representative new samples by minimizing KL divergence.
    
    Args:
        X_old (numpy.ndarray): Old memory buffer samples.
        X_new (numpy.ndarray): Incoming new samples.
        top_k (int): Number of samples to retain.
        learning_rate (float): Step size for optimization.
        num_iterations (int): Number of optimization steps.

    Returns:
        numpy.ndarray: Selected representative new samples.
    """
    
    H_old, H_new = histogram_binning(X_old), histogram_binning(X_new)
    m_n = np.random.rand(H_new.shape[0])  

    def loss_function(m_n):
        """Computes KL divergence loss for selected samples."""
        weighted_H_new = H_new * m_n[:, np.newaxis]  
        combined_H = (H_old + weighted_H_new) / 2 
        return kl_divergence(H_new, combined_H) 

    progress_bar = tqdm(total=num_iterations, desc="Optimizing Sample Selection", position=0, leave=True)

    def callback(xk):
        progress_bar.update(1)  

    result = minimize(loss_function, m_n, method="L-BFGS-B", bounds=[(0, 1)] * len(m_n), 
                      options={"maxiter": num_iterations, "ftol": 1e-10}, callback=callback)

    progress_bar.close()

    selected_indices = np.argsort(result.x)[-top_k:]

    return X_new[selected_indices] 


def update_memory_buffer(X_old, X_new_selected, memory_size=3000):
    """Updates memory buffer using strategic forgetting."""
    updated_buffer = np.vstack((X_old, X_new_selected))  

    if updated_buffer.shape[0] > memory_size:
        updated_buffer = updated_buffer[-memory_size:]

    return updated_buffer

class HierarchicalMemory:
    def __init__(self, memory_limit=5000, pyramid_factor=2, centroids_per_concept=10):
        self.memory_limit = memory_limit
        self.pyramid_factor = pyramid_factor
        self.centroids_per_concept = centroids_per_concept
        self.memory = {}  # level: [concept1, concept2, ...]

    def add_concept(self, data, level=1):
        if level not in self.memory:
            self.memory[level] = []
        self.memory[level].append(np.array(data))
        self._summarize_memory()

    def _pyramidal_allocation(self):
        levels = sorted(self.memory.keys())
        weights = np.array([1 / (self.pyramid_factor ** (lvl - 1)) for lvl in levels])
        total_weight = weights.sum()
        allocations = (weights / total_weight) * self.memory_limit
        return {lvl: int(alloc) for lvl, alloc in zip(levels, allocations)}

    def _summarize_concept(self, concept, n_samples):
        if len(concept) <= n_samples:
            return concept
        kmeans = KMeans(n_clusters=min(self.centroids_per_concept, len(concept)), random_state=42).fit(concept)
        centroids = kmeans.cluster_centers_
        distances = np.linalg.norm(concept[:, None] - centroids, axis=2)
        closest_indices = np.argmin(distances, axis=0)
        summarized = concept[closest_indices]
        return summarized

    def _summarize_memory(self):
        allocations = self._pyramidal_allocation()
        for level, concepts in self.memory.items():
            summarized_level = []
            alloc_per_concept = max(1, allocations[level] // len(concepts))
            for concept in concepts:
                summarized = self._summarize_concept(concept, alloc_per_concept)
                summarized_level.append(summarized)
            self.memory[level] = summarized_level

    def get_all_memory(self):
        all_data = []
        for level_concepts in self.memory.values():
            for concept in level_concepts:
                all_data.append(concept)
        return np.vstack(all_data) if all_data else np.empty((0,))

def scenario_design(normal_data, anomaly_data, c):
    """
    Implements Algorithm 1 to create a lifelong learning scenario.
    
    Args:
        normal_data (numpy array): The normal data points.
        anomaly_data (numpy array): The anomaly data points.
        c (int): Number of desired concepts.
    
    Returns:
        list of tuples: List of (normal_concept, anomaly_concept) pairs forming the scenario.
    """
    normal_concepts = create_phi(normal_data, c)
    anomaly_concepts = create_gamma(anomaly_data, c)
    
    scenario = match_lambda(anomaly_concepts, normal_concepts)
    
    return scenario

def evaluation_protocol(T, E, Y, model, strategy="naive", replay_buffer_size=5000, memory_size=5000, alpha=0.05):
    """
    Implements Algorithm 2: Lifelong Learning Evaluation Protocol with multiple strategies.
    
    Args:
        T (list): Sequence of N training sets.
        E (list): Sequence of N testing sets.
        Y (list): Sequence of true labels for test sets.
        model (sklearn.base.BaseEstimator): A scikit-learn-like model instance that supports `fit` and `decision_function`.
        strategy (str): Strategy for training.
        replay_buffer_size (int): Maximum size of replay buffer if applicable
        memory_size (int): Maximum memory size if applicable
        alpha (float): KS-test threshold for drift detection.

    Returns:
        numpy array: NxN results matrix R where R[i, j] is ROC-AUC of model on E[j] after learning T[i].
    """
    N = len(T)
    R = np.zeros((N, N))  

    if strategy in ["cumulative"]:
        cumulative_data = []
    
    if strategy in ["replay"]:
        replay_buffer = []

    if strategy == "SSF":
        memory_buffer = None 

    if strategy == "hierarchical":
        h_memory = HierarchicalMemory(memory_limit=memory_size, pyramid_factor=2, centroids_per_concept=10)

    for i, Ti in tqdm(enumerate(T), desc=f"Evaluating using {strategy} strategy"):
        current_model = deepcopy(model)

        # -- NAIVE --
        if strategy == "naive":
            current_model.fit(Ti)

        # -- CUMULATIVE --
        elif strategy == "cumulative":
            cumulative_data.extend(Ti.tolist())
            current_model.fit(np.array(cumulative_data)) 

        # -- REPLAY -- 
        elif strategy == "replay":
            if replay_buffer:
                combined_data = np.vstack((np.array(replay_buffer), Ti))
            else:
                combined_data = Ti

            current_model.fit(combined_data)
            replay_buffer.extend(Ti.tolist())

            if len(replay_buffer) > replay_buffer_size:
                replay_buffer = replay_buffer[-replay_buffer_size:]
        
        # -- SSF -- 
        elif strategy == "SSF":
            if memory_buffer is None:
                memory_buffer = Ti[:memory_size]  
            else:
                drift_detected = kolmogorov_smirnov_test(memory_buffer, Ti, alpha)
                if drift_detected:
                    X_new_selected = strategic_sample_selection(memory_buffer, Ti, top_k=1000)
                    memory_buffer = update_memory_buffer(memory_buffer, X_new_selected, memory_size=memory_size)
            memory_buffer = np.unique(memory_buffer, axis=0)
            current_model.fit(memory_buffer)

        # -- HIERARCHICAL --
        elif strategy == "hierarchical":

            memory_data = h_memory.get_all_memory()
            if memory_data.size == 0:
                drift_level = 1
            else:
                drift_distances = [
                    wasserstein_distance(Ti[:, d], memory_data[:, d])
                    for d in range(Ti.shape[1])
                ]
                drift_score = np.mean(drift_distances)
                print(f"drift: {drift_score}")
                
                if drift_score < 0.05:
                    drift_level = 1
                elif drift_score < 0.1:
                    drift_level = 2
                elif drift_score < 0.2:
                    drift_level = 3
                else:
                    drift_level = 4
        
            h_memory.add_concept(Ti, level=drift_level)
            summarized_memory = h_memory.get_all_memory()
            current_model.fit(summarized_memory)

        # -- Evaluation --
        for j, ((Ej_normal, Ej_anomaly), (y_normal, y_anomaly)) in enumerate(zip(E, Y)):
            test_data = np.vstack((Ej_normal, Ej_anomaly))
            test_labels = np.hstack((y_normal, y_anomaly))  
        
            scores = -current_model.decision_function(test_data)  
            R[i, j] = average_precision_score(test_labels, scores)

    return R


# Experiments

In [7]:
num_concepts = 5

X_normal = X[y == 0]  
X_anomaly = X[y == 1]

normal_concepts = create_phi(X_normal, num_concepts)
anomaly_concepts = create_gamma(X_anomaly, num_concepts)

concept_pairs = match_lambda(anomaly_concepts, normal_concepts)

T = []  
E = [] 
Y = []

for normal, anomaly in concept_pairs:

    normal_train, normal_test = train_test_split(normal, test_size=0.3, random_state=42)
    anomaly_train, anomaly_test = train_test_split(anomaly, test_size=0.3, random_state=42)  

    print(f"normal_train: {len(normal_train)}, normal_test: {len(normal_test)}, anomaly_train: {len(anomaly_train)}, anomaly_test: {len(anomaly_test)}")

    T.append(normal_train)
    E.append((normal_test, anomaly_test))

    y_normal_test = np.zeros(len(normal_test))
    y_anomaly_test = np.ones(len(anomaly_test))
    
    Y.append((y_normal_test, y_anomaly_test))

Finished creating normal concepts
Finished creating anomaly concepts
Finished matching concept pairs
normal_train: 14, normal_test: 6, anomaly_train: 777967, anomaly_test: 333415
normal_train: 71, normal_test: 31, anomaly_train: 3091, anomaly_test: 1325
normal_train: 14, normal_test: 6, anomaly_train: 26499, anomaly_test: 11357
normal_train: 14, normal_test: 6, anomaly_train: 27995, anomaly_test: 11999
normal_train: 35, normal_test: 15, anomaly_train: 422, anomaly_test: 182


# Eval

## LOF

In [10]:
R_hm = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 1it [00:00,  3.69it/s]

drift: 9.033929752227811


Evaluating using hierarchical strategy: 2it [00:00,  3.51it/s]

drift: 17.485859619105664


Evaluating using hierarchical strategy: 3it [00:00,  3.59it/s]

drift: 11.26888823583639


Evaluating using hierarchical strategy: 4it [00:01,  3.56it/s]

drift: 6.863647354128857


Evaluating using hierarchical strategy: 5it [00:01,  3.56it/s]

Lifelong ROC-AUC: 0.520468044585926, BWT: -0.028904618664700465, FWT: 0.46200064652463074


In [26]:
R_ssf = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")


luating using SSF strategy: 0it [00:00, ?it/s]
Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 273.05it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 256.08it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 276.87it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 326.53it/s]

Evaluating using SSF strategy: 5it [00:05,  1.14s/it]

Lifelong ROC-AUC: 0.7544561432893502, BWT: -0.0039135441339650965, FWT: 0.45301001764958054


In [29]:
R_naive = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 0it [00:00, ?it/s]/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/sklearn/neighbors/_lof.py:282: UserWarning: n_neighbors (20) is greater than the total number of samples (14). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn(
Evaluating using naive strategy: 2it [00:01,  1.46it/s]/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/sklearn/neighbors/_lof.py:282: UserWarning: n_neighbors (20) is greater than the total number of samples (14). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn(
Evaluating using naive strategy: 3it [00:01,  1.71it/s]/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/sklearn/neighbors/_lof.py:282: UserWarning: n_neighbors (20) is greater than the total number of samples (14). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn(
Evaluating using naive str

Lifelong PR-AUC: 0.9869316956452233, BWT: -0.01950174037502963, FWT: 0.9839654653838258


In [7]:
R_cumulative = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="cumulative")
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_cumulative)}, BWT: {BWT(R_cumulative)}, FWT: {FWT(R_cumulative)}")

Evaluating using cumulative strategy: 5it [02:10, 26.08s/it]

Lifelong ROC-AUC: 0.9238354062988797, BWT: -0.004499087364359666, FWT: 0.17379610432039533


In [30]:
R_replay = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 0it [00:00, ?it/s]/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/sklearn/neighbors/_lof.py:282: UserWarning: n_neighbors (20) is greater than the total number of samples (14). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn(
Evaluating using replay strategy: 5it [00:02,  2.03it/s]

Lifelong PR-AUC: 0.9972366305757738, BWT: -0.0025869867197455764, FWT: 0.9891627119274304


## IF

In [11]:
R_hm = evaluation_protocol(T, E, Y, IsolationForest(n_estimators=100), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 1it [00:00,  1.27it/s]

drift: 9.033929752227811


Evaluating using hierarchical strategy: 2it [00:01,  1.14it/s]

drift: 17.485859619105664


Evaluating using hierarchical strategy: 3it [00:02,  1.09it/s]

drift: 11.26888823583639


Evaluating using hierarchical strategy: 4it [00:03,  1.07it/s]

drift: 6.863647354128857


Evaluating using hierarchical strategy: 5it [00:04,  1.08it/s]

Lifelong ROC-AUC: 0.8105676014480313, BWT: -0.0024381561580483304, FWT: 0.32030028822198414


In [27]:
R_ssf = evaluation_protocol(T, E, Y,  IsolationForest(n_estimators=100), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")


luating using SSF strategy: 0it [00:00, ?it/s]
Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 281.14it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 253.28it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 237.31it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 244.61it/s]

Evaluating using SSF strategy: 5it [00:05,  1.06s/it]

Lifelong ROC-AUC: 0.6559285630296237, BWT: 0.06868127809581366, FWT: 0.804018261100736


In [31]:
R_naive = evaluation_protocol(T, E, Y, IsolationForest(n_estimators=100), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 5it [00:04,  1.24it/s]

Lifelong PR-AUC: 0.9818301936295014, BWT: -0.02259953580254522, FWT: 0.9656734349848101


In [9]:
R_cumulative = evaluation_protocol(T, E, Y, IsolationForest(n_estimators=100), strategy="cumulative")
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_cumulative)}, BWT: {BWT(R_cumulative)}, FWT: {FWT(R_cumulative)}")

Evaluating using cumulative strategy: 5it [00:07,  1.55s/it]

Lifelong ROC-AUC: 0.753897896406868, BWT: -0.0066406078135951676, FWT: 0.7719174809547474


In [32]:
R_replay = evaluation_protocol(T, E, Y, IsolationForest(n_estimators=100), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 5it [00:04,  1.13it/s]

Lifelong PR-AUC: 0.9993487268937558, BWT: -7.805230346656922e-05, FWT: 0.9751907969596898


## SGDOCSVM

In [12]:
R_hm = evaluation_protocol(T, E, Y, SGDOneClassSVM(), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 2it [00:00,  7.67it/s]

drift: 9.033929752227811
drift: 17.485859619105664


Evaluating using hierarchical strategy: 4it [00:00,  7.51it/s]

drift: 11.26888823583639
drift: 6.863647354128857


Evaluating using hierarchical strategy: 5it [00:00,  7.53it/s]

Lifelong ROC-AUC: 0.9851953661715797, BWT: -0.020546483927901583, FWT: 0.637531291800638


In [28]:
R_ssf = evaluation_protocol(T, E, Y,  SGDOneClassSVM(), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")


luating using SSF strategy: 0it [00:00, ?it/s]
Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 288.13it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 307.55it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 306.85it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 271.76it/s]

Evaluating using SSF strategy: 5it [00:02,  2.14it/s]

Lifelong ROC-AUC: 0.790790866239933, BWT: -0.004091332504534584, FWT: 0.5467376505389003


In [33]:
R_naive = evaluation_protocol(T, E, Y, SGDOneClassSVM(), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 5it [00:00,  8.49it/s]

Lifelong PR-AUC: 0.9972044916780316, BWT: -0.0030840371627078754, FWT: 0.9546643931975775


In [17]:
R_cumulative = evaluation_protocol(T, E, Y, SGDOneClassSVM(), strategy="cumulative")
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_cumulative)}, BWT: {BWT(R_cumulative)}, FWT: {FWT(R_cumulative)}")

Evaluating using cumulative strategy: 5it [00:04,  1.03it/s]


Lifelong ROC-AUC: 0.597799790906325, BWT: -0.215598947769658, FWT: 0.6579349113663381


In [34]:
R_replay = evaluation_protocol(T, E, Y, SGDOneClassSVM(), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 5it [00:00,  8.32it/s]

Lifelong PR-AUC: 0.9960680539848313, BWT: -0.001328496084602504, FWT: 0.9565455314857584


# SLAD

In [8]:
from deepod.models.tabular import SLAD

In [8]:
R_hm = evaluation_protocol(T, E, Y, SLAD(), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 43, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
epoch  1, training loss: 0.942087, time: 0.5s
epoch 10, training loss: 0.547117, time: 0.1s
epoch 20, training loss: 0.546567, time: 0.1s
epoch 30, training loss: 0.558276, time: 0.1s
epoch 40, training loss: 0.546435, time: 0.1s
epoch 50, training loss: 0.546931, time: 0.1s
epoch 60, training loss: 0.546680, time: 0.1s
epoch 70, training loss: 0.546194, time: 0.1s
epoch 80, training loss: 0.546021, time: 0.1s
epoch 90, training loss: 0.545187, time: 0.1s
epoch100, training loss: 0.546762, time: 0.1s
Start Inference on the training data...


Evaluating using hierarchical strategy: 1it [02:19, 139.10s/it]

drift: 9.033929752227811
Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 43, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
epoch  1, training loss: 0.777894, time: 0.1s
epoch 10, training loss: 0.616681, time: 0.1s
epoch 20, training loss: 0.600655, time: 0.1s
epoch 30, training loss: 0.611302, time: 0.1s
epoch 40, training loss: 0.600753, time: 0.1s
epoch 50, training loss: 0.595786, time: 0.1s
epoch 60, training loss: 0.598656, time: 0.1s
epoch 70, training loss: 0.603434, time: 0.1s
epoch 80, training loss: 0.611243, time: 0.1s
epoch 90, training loss: 0.610948, time: 0.1s
epoch100, training loss: 0.599505, time: 0.1s
Start Inference on the training data...


Evaluating using hierarchical strategy: 2it [04:36, 138.29s/it]

...

drift: 11.26888823583639
Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 43, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
epoch  1, training loss: 0.807866, time: 0.1s
epoch 10, training loss: 0.610720, time: 0.1s
epoch 20, training loss: 0.605486, time: 0.1s
epoch 30, training loss: 0.600085, time: 0.1s
epoch 40, training loss: 0.603552, time: 0.1s
epoch 50, training loss: 0.619184, time: 0.1s
epoch 60, training loss: 0.606468, time: 0.1s
epoch 70, training loss: 0.611199, time: 0.1s
epoch 80, training loss: 0.604636, time: 0.1s
epoch 90, training loss: 0.607801, time: 0.1s
epoch100, training loss: 0.598212, time: 0.1s
Start Inference on the training data...


Evaluating using hierarchical strategy: 4it [09:10, 137.33s/it]

drift: 6.863647354128857
Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 43, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
epoch  1, training loss: 0.879887, time: 0.1s
epoch 10, training loss: 0.619500, time: 0.1s
epoch 20, training loss: 0.611566, time: 0.1s
epoch 30, training loss: 0.610254, time: 0.1s
epoch 40, training loss: 0.610114, time: 0.1s
epoch 50, training loss: 0.602414, time: 0.7s
epoch 60, training loss: 0.608052, time: 0.1s
epoch 70, training loss: 0.599205, time: 0.1s
epoch 80, training loss: 0.607303, time: 0.1s
epoch 90, training loss: 0.606334, time: 0.1s
epoch100, training loss: 0.609260, time: 0.1s
Start Inference on the training data...


Evaluating using hierarchical strategy: 5it [11:29, 137.86s/it]

Lifelong ROC-AUC: 0.2581187663594956, BWT: 0.05378771477511142, FWT: 0.5442699839625316


In [42]:
R_ssf = evaluation_protocol(T, E, Y, SLAD(epochs=10), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

Evaluating using SSF strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 43, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
epoch  1, training loss: 0.705401, time: 0.0s
epoch 10, training loss: 0.583712, time: 0.0s
Start Inference on the training data...


Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 326.53it/s]


Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 43, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
epoch  1, training loss: 0.887392, time: 0.0s
epoch 10, training loss: 0.671017, time: 0.0s
Start Inference on the training data...


Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 310.39it/s]
Evaluating using SSF strategy: 2it [04:00, 120.27s/it]


IndexError: index 7 is out of bounds for axis 0 with size 6

In [36]:
R_naive = evaluation_protocol(T, E, Y, SLAD(epochs=10), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 43, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]


/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide


epoch  1, training loss: 0.701310, time: 0.4s
epoch 10, training loss: 0.591755, time: 0.0s
Start Inference on the training data...


Evaluating using naive strategy: 1it [14:21, 861.48s/it]

...

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 43, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
epoch  1, training loss: 0.792895, time: 0.0s
epoch 10, training loss: 0.578901, time: 0.0s
Start Inference on the training data...


Evaluating using naive strategy: 4it [29:14, 331.03s/it]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 43, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
epoch  1, training loss: 0.717511, time: 0.0s
epoch 10, training loss: 0.599024, time: 0.0s
Start Inference on the training data...


Evaluating using naive strategy: 5it [31:21, 376.22s/it]

Lifelong PR-AUC: 0.9797437209477171, BWT: 0.02687664847407458, FWT: 0.9157562997077079


In [37]:
R_replay = evaluation_protocol(T, E, Y, SLAD(epochs=10), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 0it [00:00, ?it/s]/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide


Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 43, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
epoch  1, training loss: 0.701310, time: 0.0s
epoch 10, training loss: 0.591755, time: 0.0s
Start Inference on the training data...


Evaluating using replay strategy: 1it [02:08, 128.78s/it]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 43, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
epoch  1, training loss: 0.978277, time: 0.1s
epoch 10, training loss: 0.601198, time: 0.1s
Start Inference on the training data...


Evaluating using replay strategy: 2it [04:14, 127.16s/it]

...

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 43, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
epoch  1, training loss: 0.917388, time: 0.1s
epoch 10, training loss: 0.581045, time: 0.1s
Start Inference on the training data...


Evaluating using replay strategy: 4it [08:49, 132.50s/it]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 43, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
epoch  1, training loss: 0.822122, time: 0.2s
epoch 10, training loss: 0.605516, time: 0.1s
Start Inference on the training data...


Evaluating using replay strategy: 5it [10:55, 131.02s/it]

Lifelong PR-AUC: 0.9615231696812983, BWT: 0.00037781483281170705, FWT: 0.928321484759897


# ICL

In [9]:
from deepod.models.tabular import ICL

In [10]:
R_hm = evaluation_protocol(T, E, Y, ICL(epochs=100), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 3
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 2/2 [00:00<00:00, 558.12it/s]

testing: 100%|██████████| 2/2 [00:00<00:00, 604.41it/s]

testing: 100%|██████████| 2/2 [00:00<00:00, 591.08it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 639.04it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 641.96it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 641.10it/s]

testing: 100%|██████████| 5221/5221 [00:08<00:00, 638.75it/s]

testing: 100%|██████████| 5221/5221 [00:08<00:00, 642.41it/s]

testing: 100%|██████████| 5221/5221 [00:08<00:00, 641.81it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 642.04it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 637.16it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 644.60it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 531.58it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 593.87it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 595.39it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 548.92it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 5

drift: 9.033929752227811
Start Training...
ensemble size: 3
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=1


testing: 100%|██████████| 3/3 [00:00<00:00, 604.31it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 610.67it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 614.61it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 643.41it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 645.13it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 645.22it/s]

testing: 100%|██████████| 5221/5221 [00:08<00:00, 642.87it/s]

testing: 100%|██████████| 5221/5221 [00:08<00:00, 633.71it/s]

testing: 100%|██████████| 5221/5221 [00:08<00:00, 640.67it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 642.77it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 644.68it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 645.02it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 526.31it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 599.64it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 591.69it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 561.04it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 6

...

drift: 11.26888823583639
Start Training...
ensemble size: 3
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=1


testing: 100%|██████████| 3/3 [00:00<00:00, 613.47it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 619.94it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 621.41it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 643.89it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 646.05it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 647.87it/s]

testing: 100%|██████████| 5221/5221 [00:08<00:00, 642.71it/s]

testing: 100%|██████████| 5221/5221 [00:08<00:00, 645.52it/s]

testing: 100%|██████████| 5221/5221 [00:08<00:00, 644.27it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 648.24it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 650.63it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 648.40it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 534.87it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 606.67it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 606.81it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 591.41it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 5

drift: 6.863647354128857
Start Training...
ensemble size: 3
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=1


testing: 100%|██████████| 3/3 [00:00<00:00, 607.69it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 593.25it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 619.60it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 645.56it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 647.08it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 647.62it/s]

testing: 100%|██████████| 5221/5221 [00:08<00:00, 644.49it/s]

testing: 100%|██████████| 5221/5221 [00:08<00:00, 642.86it/s]

testing: 100%|██████████| 5221/5221 [00:08<00:00, 638.91it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 643.89it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 645.44it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 645.70it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 529.76it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 599.64it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 593.51it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 564.74it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 5

Lifelong ROC-AUC: 0.007858929996564144, BWT: -0.0074766055384658334, FWT: 0.36056330981351264


In [ ]:
R_ssf = evaluation_protocol(T, E, Y, ICL(epochs=10), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

In [39]:
R_naive = evaluation_protocol(T, E, Y, ICL(epochs=10), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 3
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 1/1 [00:00<00:00, 496.78it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 551.59it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 540.99it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 616.32it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 606.63it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 615.58it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 601.53it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 616.57it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 614.29it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 611.77it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 616.32it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 616.75it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 608.29it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 612.76it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 612.70it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 555.02it/s]

testing: 100%|██████████| 4/4 [00:00<00

Start Training...
ensemble size: 3
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 2/2 [00:00<00:00, 566.42it/s]

testing: 100%|██████████| 2/2 [00:00<00:00, 586.57it/s]

testing: 100%|██████████| 2/2 [00:00<00:00, 580.57it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 614.91it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 616.13it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 616.10it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 599.21it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 612.63it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 611.59it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 609.00it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 616.84it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 615.86it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 611.87it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 614.42it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 613.78it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 552.35it/s]

testing: 100%|██████████| 4/4 [00:00<00

...

Start Training...
ensemble size: 3
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 1/1 [00:00<00:00, 547.34it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 565.35it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 570.73it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 618.05it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 619.28it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 619.41it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 604.27it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 618.07it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 616.44it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 618.15it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 620.32it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 619.81it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 613.55it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 620.85it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 620.42it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 560.51it/s]

testing: 100%|██████████| 4/4 [00:00<00

Start Training...
ensemble size: 3
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 1/1 [00:00<00:00, 517.88it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 541.20it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 547.92it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 619.01it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 619.70it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 609.78it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 604.62it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 618.31it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 618.70it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 615.69it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 618.06it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 615.98it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 615.28it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 616.59it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 615.48it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 554.95it/s]

testing: 100%|██████████| 4/4 [00:00<00

Lifelong PR-AUC: 0.9699575328362695, BWT: 0.006623652170684824, FWT: 0.9885452359877063


In [10]:
R_replay = evaluation_protocol(T, E, Y, ICL(epochs=10), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 3
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 1/1 [00:00<00:00, 503.34it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 525.80it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 531.87it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 629.62it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 641.89it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 641.85it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 625.95it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 647.52it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 654.28it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 644.72it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 646.21it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 645.89it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 644.77it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 644.42it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 648.06it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 569.90it/s]

testing: 100%|██████████| 4/4 [00:00<00

Start Training...
ensemble size: 3
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 2/2 [00:00<00:00, 574.64it/s]

testing: 100%|██████████| 2/2 [00:00<00:00, 587.89it/s]

testing: 100%|██████████| 2/2 [00:00<00:00, 581.17it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 642.43it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 641.57it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 641.83it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 628.21it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 646.37it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 644.83it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 643.12it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 643.61it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 645.25it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 644.95it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 646.46it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 642.83it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 567.43it/s]

testing: 100%|██████████| 4/4 [00:00<00

...

Start Training...
ensemble size: 3
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 2/2 [00:00<00:00, 555.28it/s]

testing: 100%|██████████| 2/2 [00:00<00:00, 575.11it/s]

testing: 100%|██████████| 2/2 [00:00<00:00, 574.60it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 642.04it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 640.27it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 638.66it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 632.44it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 647.82it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 644.84it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 644.49it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 646.35it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 645.99it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 645.07it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 647.00it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 641.00it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 566.09it/s]

testing: 100%|██████████| 4/4 [00:00<00

Start Training...
ensemble size: 3
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(34, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=10, out_features=50, bias=


testing: 100%|██████████| 3/3 [00:00<00:00, 636.53it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 606.11it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 606.17it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 644.06it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 641.92it/s]

testing: 100%|██████████| 5210/5210 [00:08<00:00, 641.03it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 630.58it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 644.06it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 653.11it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 643.69it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 644.84it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 647.16it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 643.87it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 644.56it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 645.20it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 555.79it/s]

testing: 100%|██████████| 4/4 [00:00<00

Lifelong PR-AUC: 0.9615458481839805, BWT: 0.0003526953194274207, FWT: 0.9328542584355585


# RCA

In [11]:
from deepod.models.tabular import RCA

In [12]:
R_hm = evaluation_protocol(T, E, Y, RCA(epochs=10), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 550.22it/s]

100%|██████████| 10/10 [00:01<00:00,  5.88it/s]

100%|██████████| 10/10 [00:47<00:00,  4.76s/it]

100%|██████████| 10/10 [00:01<00:00,  5.85it/s]

100%|██████████| 10/10 [00:00<00:00, 345.20it/s]

100%|██████████| 10/10 [00:00<00:00, 1079.78it/s]
Evaluating using hierarchical strategy: 1it [00:51, 51.55s/it]

drift: 9.033929752227811
Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, o


100%|██████████| 10/10 [00:00<00:00, 364.69it/s]

100%|██████████| 10/10 [00:01<00:00,  5.86it/s]

100%|██████████| 10/10 [00:47<00:00,  4.75s/it]

100%|██████████| 10/10 [00:01<00:00,  5.88it/s]

100%|██████████| 10/10 [00:00<00:00, 344.37it/s]

100%|██████████| 10/10 [00:00<00:00, 1091.36it/s]
Evaluating using hierarchical strategy: 2it [01:43, 51.73s/it]

...

drift: 11.26888823583639
Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, o


100%|██████████| 10/10 [00:00<00:00, 365.95it/s]

100%|██████████| 10/10 [00:01<00:00,  5.85it/s]

100%|██████████| 10/10 [00:47<00:00,  4.75s/it]

100%|██████████| 10/10 [00:01<00:00,  5.88it/s]

100%|██████████| 10/10 [00:00<00:00, 344.16it/s]

100%|██████████| 10/10 [00:00<00:00, 1089.17it/s]
Evaluating using hierarchical strategy: 4it [03:26, 51.54s/it]

drift: 6.863647354128857
Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, o


100%|██████████| 10/10 [00:00<00:00, 360.06it/s]

100%|██████████| 10/10 [00:01<00:00,  5.88it/s]

100%|██████████| 10/10 [00:47<00:00,  4.75s/it]

100%|██████████| 10/10 [00:01<00:00,  5.87it/s]

100%|██████████| 10/10 [00:00<00:00, 340.55it/s]

100%|██████████| 10/10 [00:00<00:00, 1099.42it/s]
Evaluating using hierarchical strategy: 5it [04:17, 51.55s/it]

Lifelong ROC-AUC: 0.9300602880705339, BWT: -7.23431765690119e-05, FWT: 0.598798912880806


In [ ]:
R_ssf = evaluation_protocol(T, E, Y, RCA(epochs=10), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

In [12]:
R_naive = evaluation_protocol(T, E, Y, RCA(epochs=10), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 1066.60it/s]

100%|██████████| 10/10 [00:47<00:00,  4.80s/it]

100%|██████████| 10/10 [00:00<00:00, 50.34it/s][A

100%|██████████| 10/10 [00:01<00:00,  6.23it/s]

100%|██████████| 10/10 [00:01<00:00,  5.91it/s]

100%|██████████| 10/10 [00:00<00:00, 277.88it/s]
Evaluating using naive strategy: 1it [00:51, 51.87s/it]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 560.42it/s]

100%|██████████| 10/10 [00:47<00:00,  4.73s/it]

100%|██████████| 10/10 [00:00<00:00, 50.38it/s][A

100%|██████████| 10/10 [00:01<00:00,  6.22it/s]

100%|██████████| 10/10 [00:01<00:00,  5.90it/s]

100%|██████████| 10/10 [00:00<00:00, 273.18it/s]
Evaluating using naive strategy: 2it [01:43, 51.50s/it]

...

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 1074.77it/s]

100%|██████████| 10/10 [00:47<00:00,  4.73s/it]

100%|██████████| 10/10 [00:00<00:00, 49.44it/s]

100%|██████████| 10/10 [00:01<00:00,  6.16it/s]

100%|██████████| 10/10 [00:01<00:00,  5.85it/s]

100%|██████████| 10/10 [00:00<00:00, 272.90it/s]
Evaluating using naive strategy: 4it [03:25, 51.34s/it]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 1009.02it/s]

100%|██████████| 10/10 [00:47<00:00,  4.77s/it]

100%|██████████| 10/10 [00:00<00:00, 50.00it/s][A

100%|██████████| 10/10 [00:01<00:00,  6.18it/s]

100%|██████████| 10/10 [00:01<00:00,  5.85it/s]

100%|██████████| 10/10 [00:00<00:00, 269.96it/s]
Evaluating using naive strategy: 5it [04:17, 51.44s/it]

Lifelong PR-AUC: 0.9858790768979343, BWT: 1.4620844117740894e-05, FWT: 0.9160177796181503


In [13]:
R_replay = evaluation_protocol(T, E, Y, RCA(epochs=10), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 1075.46it/s]

100%|██████████| 10/10 [00:47<00:00,  4.76s/it]

100%|██████████| 10/10 [00:00<00:00, 49.36it/s]

100%|██████████| 10/10 [00:01<00:00,  6.18it/s]

100%|██████████| 10/10 [00:01<00:00,  5.85it/s]

100%|██████████| 10/10 [00:00<00:00, 272.34it/s]
Evaluating using replay strategy: 1it [00:51, 51.55s/it]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 530.85it/s]

100%|██████████| 10/10 [00:47<00:00,  4.76s/it]

100%|██████████| 10/10 [00:00<00:00, 49.32it/s]

100%|██████████| 10/10 [00:01<00:00,  6.20it/s]

100%|██████████| 10/10 [00:01<00:00,  5.85it/s]

100%|██████████| 10/10 [00:00<00:00, 272.80it/s]
Evaluating using replay strategy: 2it [01:43, 51.56s/it]

...

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 514.83it/s]

100%|██████████| 10/10 [00:47<00:00,  4.74s/it]

100%|██████████| 10/10 [00:00<00:00, 49.95it/s][A

100%|██████████| 10/10 [00:01<00:00,  6.24it/s]

100%|██████████| 10/10 [00:01<00:00,  5.90it/s]

100%|██████████| 10/10 [00:00<00:00, 277.00it/s]
Evaluating using replay strategy: 4it [03:25, 51.45s/it]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=43, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 363.32it/s]

100%|██████████| 10/10 [00:47<00:00,  4.74s/it]

100%|██████████| 10/10 [00:00<00:00, 50.47it/s][A

100%|██████████| 10/10 [00:01<00:00,  6.26it/s]

100%|██████████| 10/10 [00:01<00:00,  5.91it/s]

100%|██████████| 10/10 [00:00<00:00, 278.77it/s]
Evaluating using replay strategy: 5it [04:17, 51.48s/it]

Lifelong PR-AUC: 0.9858643959540109, BWT: -2.683552163018632e-06, FWT: 0.9160174323186847


# RDP

In [14]:
from deepod.models.tabular import RDP

In [14]:
R_hm = evaluation_protocol(T, E, Y, RDP(epochs=10), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=43, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.004941, time: 0.0s
epoch 10, training loss: 0.008679, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 2/2 [00:00<00:00, 925.69it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 1056.08it/s]

testing: 100%|██████████| 5221/5221 [00:04<00:00, 1060.44it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 1055.17it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 820.32it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 859.31it/s]
Evaluating using hierarchical strategy: 1it [00:05,  5.74s/it]

drift: 9.033929752227811
Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=43, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.036421, time: 0.0s
epoch 10, training loss: 0.001843, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 3/3 [00:00<00:00, 975.65it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 1061.96it/s]

testing: 100%|██████████| 5221/5221 [00:04<00:00, 1058.19it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 1062.28it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 826.95it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 868.21it/s]
Evaluating using hierarchical strategy: 2it [00:11,  5.76s/it]

...

drift: 11.26888823583639
Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=43, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.031406, time: 0.0s
epoch 10, training loss: 0.004282, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 3/3 [00:00<00:00, 983.50it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 1060.65it/s]

testing: 100%|██████████| 5221/5221 [00:05<00:00, 1038.89it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 1056.60it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 816.22it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 876.19it/s]
Evaluating using hierarchical strategy: 4it [00:23,  5.81s/it]

drift: 6.863647354128857
Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=43, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.009494, time: 0.0s
epoch 10, training loss: 0.002171, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 3/3 [00:00<00:00, 957.82it/s]

testing: 100%|██████████| 189/189 [00:00<00:00, 1056.89it/s]

testing: 100%|██████████| 5221/5221 [00:04<00:00, 1062.08it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 1048.76it/s]

testing: 100%|██████████| 3/3 [00:00<00:00, 809.82it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 881.16it/s]
Evaluating using hierarchical strategy: 5it [00:28,  5.78s/it]

Lifelong ROC-AUC: 0.8714016611717592, BWT: 0.08574400357883394, FWT: 0.5969335576215077


In [ ]:
R_ssf = evaluation_protocol(T, E, Y, RDP(epochs=10), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

In [15]:
R_naive = evaluation_protocol(T, E, Y, RDP(epochs=10), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=43, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000000, time: 0.0s
epoch 10, training loss: 0.000127, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 1/1 [00:00<00:00, 835.52it/s]

testing: 100%|██████████| 5210/5210 [00:04<00:00, 1062.88it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 1030.60it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 1061.14it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 1054.78it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 887.73it/s]
Evaluating using naive strategy: 1it [00:05,  5.69s/it]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=43, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.005491, time: 0.0s
epoch 10, training loss: 0.008794, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 2/2 [00:00<00:00, 958.92it/s]

testing: 100%|██████████| 5210/5210 [00:04<00:00, 1054.61it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 1023.16it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 1053.20it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 1052.44it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 911.21it/s]
Evaluating using naive strategy: 2it [00:11,  5.73s/it]

...

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=43, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000000, time: 0.0s
epoch 10, training loss: 0.000643, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 1/1 [00:00<00:00, 648.77it/s]

testing: 100%|██████████| 5210/5210 [00:04<00:00, 1065.73it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 1028.35it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 1053.07it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 1054.60it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 928.20it/s]
Evaluating using naive strategy: 4it [00:22,  5.69s/it]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=43, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000000, time: 0.0s
epoch 10, training loss: 0.000032, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 1/1 [00:00<00:00, 795.88it/s]

testing: 100%|██████████| 5210/5210 [00:04<00:00, 1064.27it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 1023.08it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 1053.06it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 1049.13it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 889.80it/s]
Evaluating using naive strategy: 5it [00:28,  5.70s/it]

Lifelong PR-AUC: 0.9856491053888253, BWT: 0.0005282834578947959, FWT: 0.9160205394392686


In [16]:
R_replay = evaluation_protocol(T, E, Y, RDP(epochs=10), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=43, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000000, time: 0.0s
epoch 10, training loss: 0.000127, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 1/1 [00:00<00:00, 893.74it/s]

testing: 100%|██████████| 5210/5210 [00:05<00:00, 1035.87it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 1013.07it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 1050.59it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 1063.11it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 912.95it/s]
Evaluating using replay strategy: 1it [00:05,  5.81s/it]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=43, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.004896, time: 0.0s
epoch 10, training loss: 0.007682, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 2/2 [00:00<00:00, 935.92it/s]

testing: 100%|██████████| 5210/5210 [00:04<00:00, 1063.02it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 1020.43it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 1045.11it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 1053.69it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 906.68it/s]
Evaluating using replay strategy: 2it [00:11,  5.76s/it]

...

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=43, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.004558, time: 0.0s
epoch 10, training loss: 0.005264, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 2/2 [00:00<00:00, 925.38it/s]

testing: 100%|██████████| 5210/5210 [00:04<00:00, 1062.09it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 1023.44it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 1053.76it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 1053.12it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 889.38it/s]
Evaluating using replay strategy: 4it [00:22,  5.73s/it]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=43, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.017191, time: 0.0s
epoch 10, training loss: 0.003446, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 3/3 [00:00<00:00, 1035.37it/s]

testing: 100%|██████████| 5210/5210 [00:04<00:00, 1055.81it/s]

testing: 100%|██████████| 22/22 [00:00<00:00, 1020.30it/s]

testing: 100%|██████████| 178/178 [00:00<00:00, 1058.10it/s]

testing: 100%|██████████| 188/188 [00:00<00:00, 1054.04it/s]

testing: 100%|██████████| 4/4 [00:00<00:00, 896.55it/s]
Evaluating using replay strategy: 5it [00:28,  5.75s/it]

Lifelong PR-AUC: 0.985353005392868, BWT: 3.835378325880434e-05, FWT: 0.9160214245775121
